In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK 2.0 Remote Training for lightning model

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vertex_ai_sdk/remote_training_lightning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vertex_ai_sdk/remote_training_lightning.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
    <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/vertex_ai_sdk/remote_training_lightning.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo">Open in Vertex AI Workbench
    </a>
</table>

## Overview

This tutorial demonstrates how to use Vertex AI SDK 2.0 for remote model training of a local model training job for OSS ML frameworks.

### Objective

In this tutorial, you learn to use `Vertex AI SDK 2.0` to remotely training models of various ML frameworks as a local (on-prem) training job.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Training`
- `Vertex AI Remote Training`

The steps performed include:

- Download and split the dataset
- Perform transformations as a Vertex AI remote training.
- For scikit-learn, PyTorch, TensorFlow, PyTorch Lightning
    - Train the model remotely.
    - Uptrain the pretrained model remotely.
    - Evaluate both the pretrained and uptrained model.

**Local-to-remote training**

```
import vertexai
from my_module import MyModelClass

vertexai.init(project="my-project", location="my-location", staging_bucket="gs://my-bucket")

# Switch to remote mode
vertexai.preview.init(remote=True)

# Wrap the model class with `vertexai.preview.remote`
MyModelClass = vertexai.preview.remote(MyModelClass)

# Instantiate the class
model = MyModelClass(...)

# Optional set training config
model.fit.vertex.remote_config.display_name = "MyModelClass-remote-training"
model.fit.vertex.remote_config.staging_bucket = "gs://my-bucket"

# This `fit` call will be executed remotely
model.fit(...)
```

*Remote training supported OSS ML frameworks*
1.  scikit-learn
2.  TensorFlow
3.  PyTorch
4.  Pytorch Lightning
5.  Custom model


---

**Uptraining**
```
...
model = MyModelClass(...)
model.fit(...)

# Save the trained model to Model Registry
registered_model = vertexai.preview.register(model)

# The model can be loaded to a new (or current) local runtime
loaded_model = vertexai.preview.from_pretrained("registered-model-resource-id")

# Loaded model can cuntinue perform local-to-remote training
loaded_model.fit(...)

```

*Remote training supported OSS ML frameworks*
1.  scikit-learn
2.  TensorFlow
3.  Custom model
4.  PyTorch



---

**GPU Training**
```
...
model = MyModelClass(...)

# Set enable_cuda to True to enable GPU training.
model.fit.vertex.remote_config.enable_cuda = True

# (Optional) Training image and compute resources will be automatically
# handled by Vertex, but you can also config by yourself.
model.fit.vertex.remote_config.container_uri = "your-cuda-image"
model.fit.vertex.remote_config.machine_type = "a2-highgpu-8g"
model.fit.vertex.remote_config.accelerator_type = "NVIDIA_TESLA_A100"
model.fit.vertex.remote_config.accelerator_count = 8

# Model will be trained remotely using GPU
model.fit(...)
```

*GPU remote training supported OSS ML frameworks*
1.  TensorFlow
2.  PyTorch

### Dataset

This tutorial uses the <a href="https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html">IRIS dataset</a>, which predicts the iris species.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform[preview]
! pip3 install --upgrade --quiet scikit-learn
! pip3 install --upgrade --quiet lightning

### Colab only: Uncomment the following cell to restart the kernel

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### Import libraries and define constants

In [ ]:
import vertexai
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
REMOTE_JOB_NAME = "remote-scalar"
REMOTE_JOB_BUCKET = f"{BUCKET_URI}/{REMOTE_JOB_NAME}"

vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=REMOTE_JOB_BUCKET,
)

## Prepare the dataset

Now load the Iris dataset and split the data into train, retrain and test sets.

In [ ]:
dataset = load_iris()

X, X_retrain, y, y_retrain = train_test_split(
    dataset.data, dataset.target, test_size=0.60, random_state=42
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)


transformer = StandardScaler()
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)
X_retrain = transformer.transform(X_retrain)

## PyTorch Lightning

### Remote training

First, train a PyTorch Ligtning model as a remote training job:

- Reinitialize Vertex AI for remote training.
- Set Trainer for the remote training job.
- Invoke Trainer locally which will launch the remote training job.

In [ ]:
# Switch to local mode for testing
vertexai.preview.init(remote=True)

import lightning.pytorch as pl
import torch

# Wrap classes to enable Vertex remote execution
pl.Trainer = vertexai.preview.remote(pl.Trainer)


# prepare data loaders
train_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.tensor(X_train).to(torch.float32),
        torch.tensor(y_train),
    ),
    batch_size=10,
    shuffle=True,
)

val_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.tensor(X_test).to(torch.float32),
        torch.tensor(y_test),
    ),
    batch_size=10,
)


# define the model
class LitLogisticRegression(pl.LightningModule):
    def __init__(self, input_size: int, output_size: int):
        super().__init__()
        self.linear = torch.nn.Linear(input_size, output_size)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        return self.softmax(self.linear(x))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.cross_entropy(y_hat, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.cross_entropy(y_hat, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.05)

    def predict(self, X):
        X = torch.tensor(X).to(torch.float32)
        with torch.no_grad():
            pred = torch.argmax(self(X), dim=1)
        return pred


model = LitLogisticRegression(4, 3)

# set up the trainer and training config
trainer = pl.Trainer(max_epochs=100, accelerator="cpu")

# Train model on Vertex
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

### Local evaluation

Next, evaluate the pretrained version of the model, and compare the results.

In [ ]:
from sklearn.metrics import accuracy_score

# Switch to local mode for testing
vertexai.preview.init(remote=False)

# Evaluate model's accuracy score
print(f"Train accuracy: {accuracy_score(y_train, model.predict(X_train))}")
print(f"Test accuracy: {accuracy_score(y_test, model.predict(X_test))}")

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
import os

delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}